<a href="https://colab.research.google.com/github/hyaochn/RUP-Net/blob/master/RUP_Net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Residual U-Net + Pixel De-convolution Network**



1.   Import library
2.   Mount Google Drive to load dataset. Att: /X, /Y to load correctly.


In [68]:
%tensorflow_version 2.x
import tensorflow as tf
print("tensorflow version: ",tf.__version__)

import h5py
import numpy as np

import tensorflow.keras as keras


tensorflow version:  2.1.0


In [3]:
from google.colab import drive
drive.mount('gdrive')
#!ls "gdrive/My Drive/Deep-Learning/Data"


!pip install tensorflow_io

import tensorflow_io as tfio
print("tensorflow_io version: ",tfio.__version__)

Drive already mounted at gdrive; to attempt to forcibly remount, call drive.mount("gdrive", force_remount=True).
tensorflow_io version:  0.12.0


In [0]:
root_dir="gdrive/My Drive/Deep-Learning/"
train_fn=root_dir+"Data/train.h5"
test_fn=root_dir+"Data/test.h5"

#!ls "$train_fn"

#x_train, y_train= tfio.IODataset.from_hdf5(train_fn, dataset='/X'), tfio.IODataset.from_hdf5(train_fn, dataset='/Y')
#x_test, y_test= tfio.IODataset.from_hdf5(test_fn, dataset='/X'), tfio.IODataset.from_hdf5(test_fn, dataset='/Y')

#conf=tf.io.read_file(root_dir+"Configures/train.csv")


In [95]:
#last channel axis is also number of class (1+1=2)
#image_shape=x_train.element_spec.shape
#print(image_shape)

BATCH_SIZE=2
EPOCH=5

train_f=h5py.File(train_fn,'r')
x_train, y_train=np.array(train_f['X']), np.array(train_f['Y'])

SHUFFLE_SIZE=BATCH_SIZE*np.prod(y_train.shape)

y_train=tf.one_hot(y_train, 2, 4)
train_set=tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_set=train_set.batch(BATCH_SIZE).shuffle(SHUFFLE_SIZE).prefetch(tf.data.experimental.AUTOTUNE)

print(np.amin(x_train))
print(np.amax(x_train))
print(train_set.element_spec)
print(y_train.shape)


0.0
4605.0
(TensorSpec(shape=(None, 64, 64, 64, 1), dtype=tf.float32, name=None), TensorSpec(shape=(None, 64, 64, 64, 2), dtype=tf.int32, name=None))
(64, 64, 64, 64, 2)


In [0]:
#  keras.layers.BatchNormalization(),
model = keras.Sequential([
  keras.layers.InputLayer(input_shape=x_train.shape[1:]),
  keras.layers.Conv3D(2, kernel_size=(3,3,3), padding="same",activation=keras.activations.relu)
])

model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.CategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [99]:
model.fit(train_set, epochs=EPOCH)


Train for 32 steps
Epoch 1/5
32/32 [==============================] - 6s 186ms/step - loss: 107.5638 - accuracy: 0.8335
Epoch 2/5
32/32 [==============================] - 1s 33ms/step - loss: 111.3637 - accuracy: 0.8442
Epoch 3/5
32/32 [==============================] - 1s 27ms/step - loss: 112.3105 - accuracy: 0.8503
Epoch 4/5
32/32 [==============================] - 1s 23ms/step - loss: 112.4167 - accuracy: 0.8539
Epoch 5/5
32/32 [==============================] - 1s 24ms/step - loss: 113.5958 - accuracy: 0.8622
